## Doctor appointment booking System

This chat agent can interact with users and perform the following tasks:

    1. Select a doctor and choose a time slot for booking

    2. Cancel a booked appointment

    3. Reschedule a booked appointment based on availability

    4. Search for doctor details filtered by name, availability, specialty, and location

### 1. Connect with Gemini API

connect to the llm using google api key.

In [1]:
# connect with gemini api
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

In [2]:
# initialize llm chat model
from langchain_google_genai import ChatGoogleGenerativeAI
from google.genai.types import GenerateContentConfig

system_instruction = """
                        You are a phone-agent assistant.  
                        
                        Keep all messages very short—just a single question or one answer.  
                        
                        Act as if you’re on a live call: no long explanations or chit-chat.  
                        
                        Always reference prior chat context and stay aligned with the conversation.  
                        
                    """

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-preview-05-20",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    config=GenerateContentConfig(
        system_instruction=system_instruction,
    ),
    # other params...
)

### 4. Create DataBase and insert sample data to that

This make a database and add some sample data 

In [3]:
DB_PATH = "hospital3.db"

In [ ]:
import sqlite3
from datetime import datetime

# Step 1: Connect
DB_PATH = "hospital3.db"
conn = sqlite3.connect(DB_PATH) 
cursor = conn.cursor()

# Step 2: Create tables
cursor.execute("""
CREATE TABLE Doctor (
    Doctor_ID INTEGER PRIMARY KEY AUTOINCREMENT,
    Doctor_Name TEXT,
    Specialization TEXT,
    Location TEXT,
    Rating REAL
)
""")

cursor.execute("""
CREATE TABLE Patient (
    Patient_ID INTEGER PRIMARY KEY AUTOINCREMENT,
    Patient_Name TEXT,
    Age INTEGER
)
""")

cursor.execute("""
CREATE TABLE Appointment (
    Appointment_ID INTEGER PRIMARY KEY AUTOINCREMENT,
    Appointment_Time TEXT,
    Appointment_Date TEXT,
    Doctor_ID INTEGER,
    Patient_ID INTEGER,
    FOREIGN KEY (Doctor_ID) REFERENCES Doctor(Doctor_ID),
    FOREIGN KEY (Patient_ID) REFERENCES Patient(Patient_ID)
)
""")

# Insert data into Doctor and Patient tables
doctors = [
    ("Dr. Alice Smith", "Cardiology", "New York", 4.7),
    ("Dr. Bob Johnson", "Neurology", "Los Angeles", 4.5),
    ("Dr. Clara Lee", "Pediatrics", "Chicago", 4.8)
]

patients = [
    ("John Doe", 30),
    ("Emily Davis", 25),
    ("Michael Brown", 45)
]

cursor.executemany("INSERT INTO Doctor (Doctor_Name, Specialization, Location, Rating) VALUES (?, ?, ?, ?)", doctors)
cursor.executemany("INSERT INTO Patient (Patient_Name, Age) VALUES (?, ?)", patients)

# Retrieve IDs
cursor.execute("SELECT Doctor_ID FROM Doctor ORDER BY Doctor_ID")
doctor_ids = [row[0] for row in cursor.fetchall()]

cursor.execute("SELECT Patient_ID FROM Patient ORDER BY Patient_ID")
patient_ids = [row[0] for row in cursor.fetchall()]

# Insert into Appointment table
appointments = [
    ("10:00", "2025-06-06", doctor_ids[0], patient_ids[0]),
    ("11:30", "2025-06-06", doctor_ids[1], patient_ids[1]),
    ("14:00", "2025-06-07", doctor_ids[2], patient_ids[2])
]

cursor.executemany("""
    INSERT INTO Appointment (Appointment_Time, Appointment_Date, Doctor_ID, Patient_ID)
    VALUES (?, ?, ?, ?)
""", appointments)

# Step 4: Commit and verify
conn.commit()

# Print the data
for row in cursor.execute("SELECT * FROM Appointment"):
    print(row)

# Close connection
conn.close()

### 5. Tools for searching a Doctor and book an Appointment

In [4]:
from langchain_core.tools import tool
from typing import Optional, Dict, List
import sqlite3
import json

# Tool to search for a doctor by name
@tool
def search_for_doctor(name: Optional[str] = None) -> List[Dict]:
    """
    Search for doctors by name (or return all if name is None).

    Args:
        name (Optional[str]): The partial or full name of the doctor.

    Returns:
        List[Dict]: Doctors matching the search criteria.
    """
    conn = sqlite3.connect(DB_PATH)
    conn.row_factory = sqlite3.Row
    cursor = conn.cursor()

    if name:
        cursor.execute("""
            SELECT * FROM Doctor WHERE Doctor_Name LIKE ?
        """, (f"%{name}%",))
    else:
        cursor.execute("SELECT * FROM Doctor")

    results = [dict(row) for row in cursor.fetchall()]
    conn.close()
    return json.dumps(results)

# Tool to check a doctor's availability at a given date and time
@tool
def check_doctor_availability(doctor_id: int, date: str, time: str) -> Dict:
    """
    Check if a doctor is available at a given date and time.

    Args:
        doctor_id (int): The ID of the doctor.
        date (str): Date of the appointment in 'YYYY-MM-DD' format.
        time (str): Time of the appointment (e.g., '10:00').

    Returns:
        Dict: {"available": True/False}
    """
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    cursor.execute("""
        SELECT 1 FROM Appointment
        WHERE Doctor_ID = ? AND Appointment_Date = ? AND Appointment_Time = ?
    """, (doctor_id, date, time))

    result = cursor.fetchone()
    available = result is None
    conn.close()

    return json.dumps({"available": available})

# Tool for book an appoinment and add to the database
@tool
def book_appointment(user_id: int, doctor_id: int, date: str, time: str) -> Dict:
    """
    Book an appointment with a doctor if available at the given date and time.

    Args:
        user_id (int): The ID of the patient.
        doctor_id (int): The ID of the doctor.
        date (str): Appointment date in 'YYYY-MM-DD' format.
        time (str): Appointment time in 'HH:MM' format.

    Returns:
        Dict: {"success": True/False, "message": str}
    """
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    # Check if doctor is available
    cursor.execute("""
        SELECT 1 FROM Appointment
        WHERE Doctor_ID = ? AND Appointment_Date = ? AND Appointment_Time = ?
    """, (doctor_id, date, time))
    result = cursor.fetchone()

    if result:
        conn.close()
        return {"success": False, "message": "Doctor is not available at this date and time."}

    # Book appointment
    cursor.execute("""
        INSERT INTO Appointment (Appointment_Time, Appointment_Date, Doctor_ID, Patient_ID)
        VALUES (?, ?, ?, ?)
    """, (time, date, doctor_id, user_id))
    conn.commit()
    conn.close()

    return json.dumps({"success": True, "message": "Appointment booked successfully."})

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from datetime import datetime

book_doctor_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You help users book new doctor appointments. Use tools to find doctors (by name, specialty, or location), "
            "check availability, and book only after confirming both availability and user intent. "
            "Do not proceed without tool-confirmed availability.\n\n"
            "Tools:\n"
            "- `search_for_doctor`: Find doctors by user criteria.\n"
            "- `check_doctor_availability`: Confirm if a doctor is free.\n"
            "- `book_appointment`: Book only after confirmation.\n"
            "- `CompleteOrEscalate`: Use for unrelated requests (e.g., cancel, reschedule, insurance).\n\n"
            "Current time: {time}.\n"
            "Use `CompleteOrEscalate` for requests like:\n"
            "- Reschedule or cancel\n"
            "- Insurance or non-appointment tasks\n"
            "- User already handled it themselves",
        ),
        ("placeholder", "{messages}"),
    ]
).partial(time=datetime.now())

In [6]:
from langgraph.prebuilt import create_react_agent

book_appointment_tools = [check_doctor_availability, search_for_doctor, book_appointment]

agent = create_react_agent(
    model=llm,  
    tools=book_appointment_tools,  
    prompt=book_doctor_prompt
)

In [8]:
# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "I want to book an appointment. wit doctor alice"}]}
)

{'messages': [HumanMessage(content='I want to book an appointment. wit doctor alice', additional_kwargs={}, response_metadata={}, id='41dcf77f-1173-4b30-adc7-d113487134c5'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'search_for_doctor', 'arguments': '{"name": "alice"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'models/gemini-2.5-flash-preview-05-20', 'safety_ratings': []}, id='run--173ebc8c-664d-4322-9463-0f5170cac5a9-0', tool_calls=[{'name': 'search_for_doctor', 'args': {'name': 'alice'}, 'id': '8783a59d-9a15-45e2-844d-090d229d095e', 'type': 'tool_call'}], usage_metadata={'input_tokens': 595, 'output_tokens': 17, 'total_tokens': 667, 'input_token_details': {'cache_read': 0}}),
  ToolMessage(content='[{"Doctor_ID": 1, "Doctor_Name": "Dr. Alice Smith", "Specialization": "Cardiology", "Location": "New York", "Rating": 4.7}]', name='search_for_doctor', id='aab0fb40-3402-4223-9e60-65